In [1]:
import nltk
from nltk.corpus import treebank
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
from nltk.tokenize import word_tokenize


In [2]:
nltk.download('treebank')


[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\vmakh\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\vmakh\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
sentences = treebank.tagged_sents(tagset='universal')


In [5]:
train_data = sentences[:3000]
test_data = sentences[3000:]


In [6]:
train_data[:10]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

In [7]:
unigram_tagger = UnigramTagger(train_data)
print(unigram_tagger.evaluate(test_data))


C:\Users\vmakh\AppData\Local\Temp\ipykernel_7480\3836260245.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(unigram_tagger.evaluate(test_data))


0.8761493632635441


In [8]:
model = unigram_tagger._context_to_tag

for i, (context, tag) in enumerate(list(model.items())[:10]):
    print(f"Context: {context}, Tag: {tag}")

Context: anti-abortion, Tag: ADJ
Context: eat, Tag: VERB
Context: Rep., Tag: NOUN
Context: NIH, Tag: NOUN
Context: opens, Tag: VERB
Context: Strother, Tag: NOUN
Context: N.C, Tag: NOUN
Context: screenwriters, Tag: NOUN
Context: francs, Tag: NOUN
Context: controls, Tag: NOUN


In [9]:
type(unigram_tagger)

nltk.tag.sequential.UnigramTagger

In [10]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
print(bigram_tagger.evaluate(test_data))


C:\Users\vmakh\AppData\Local\Temp\ipykernel_7480\893261768.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(bigram_tagger.evaluate(test_data))


0.8799481977120657


In [11]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
print(trigram_tagger.evaluate(test_data))



C:\Users\vmakh\AppData\Local\Temp\ipykernel_7480\4093495547.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(trigram_tagger.evaluate(test_data))


0.879516511979279


In [12]:
from nltk.tag import TrigramTagger 

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data) 


C:\Users\vmakh\AppData\Local\Temp\ipykernel_7480\154194755.py:14: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tag.evaluate(test_data)


0.879516511979279

In [13]:
nltk.download('names')


[nltk_data] Downloading package names to
[nltk_data]     C:\Users\vmakh\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [17]:
from nltk.tag import SequentialBackoffTagger, DefaultTagger
from nltk.corpus import names
from nltk.corpus import treebank

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, train_sents):
        SequentialBackoffTagger.__init__(self)
        self.name_set = set([n.lower() for n in names.words()])
        #self.backoff_tagger = DefaultTagger('NN')
        self.backoff_tagger = BigramTagger(train_sents, backoff=DefaultTagger('NN'))
        self._train(train_sents)

    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
            return 'NOUN'  # or 'NOUN' depending on the tagset

        else:
            return None

    def _train(self, tagged_corpus, *args, **kwargs):
        self._taggers = [self] + [self.backoff_tagger]

sentences = treebank.tagged_sents(tagset='universal')
train_data = sentences[:3000]
test_data = sentences[3000:]

names_tagger = NamesTagger(train_data)

accuracy = names_tagger.evaluate(test_data)
print("Accuracy:", accuracy)













C:\Users\vmakh\AppData\Local\Temp\ipykernel_7480\1656317430.py:30: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = names_tagger.evaluate(test_data)


Accuracy: 0.19892078566803367
